# **BMI_status (ETL)**

## **BMI is an important criterion for determination of health issues. In this ETL, data about a person's height and weight is extracted from various sources, the units are changed into metric, BMI is calculated  and weight status is found out and all this data is loaded as a single csv file.**

Steps Involved: 
- Import the required modules and functions
- Setting paths for temporary, log and target files
- Creating user defined funtions to extract csv, json and specific xml files
- Read CSV and JSON file types. 
- Extract data from the above file types. 
- Transform data. 
- Save the transformed data in a ready-to-load format which data engineers can use to load into an RDBMS. 


## **Importing the required modules and functions**

In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

## **Setting Paths**

In [2]:
tmpfile    = "temp.tmp"                        # file used to store all extracted data
logfile    = "logfile.txt"                     # all event logs will be stored in this file
targetfile = "./output/BMI_status.csv"         # file where transformed data is stored

## **Functions for Extraction**

### **Function to extract csv**

In [3]:
def extract_from_csv(file_to_process):
    df = pd.read_csv(file_to_process)
    return df

### **Function to extract json**

In [4]:
def extract_from_json(file_to_process):
    df = pd.read_json(file_to_process,lines=True)
    return df

### **Function to extract xml**

In [5]:
def extract_from_xml(file_to_process):
    df = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        df = df.append({"name":name, "height":height, "weight":weight}, ignore_index=True)
    return df

### **Function to extract all data into a single dataframe**

In [6]:
def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        extracted_data = extracted_data.drop_duplicates()
    return extracted_data

## **Functions for Transformation**


**The transform function does the following tasks.**

1.  **Convert height which is in inches to millimeter**
2.  **Convert weight which is in pounds to kilograms**
3.  **Create a new column BMI**
4.  **Create a new column weight_status**

In [7]:
def transform(data):
        
        #Convert inches to meters and round off to two decimals(one inch is 0.0254 meters)
        data['height'] = round(data.height * 0.0254,2)
        
    
        #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
        data['weight'] = round(data.weight * 0.45359237,2)
        
        #creating a new column BMI. BMI = weight in kg/(height in meters)^2
        data['BMI'] = round(data['weight']/(data['height']**2),2)
        
        #creating a function to map BMI score to weight status (underweight, normal,overweight, obese)
        def weight_status(BMI):
            if (BMI <18.5):
                return 'underweight'
            elif (BMI >= 18.5 and BMI < 25.0):
                return 'normal'
            elif (BMI >=25.0 and BMI< 30.0):
                return 'overweight'
            elif (BMI >=30.0):
                return 'obese'
            
    
        #creating anew column weight_status  
        data['weight_status'] = data['BMI'].apply(weight_status)
        
        return data

## **Function for Loading**

In [8]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile, index=False)  

## **Function for Logging**

In [9]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

# **Running ETL Process**

In [10]:
log("ETL Job Started")

In [11]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30
5,jack,68.70,123.30
6,tom,69.80,141.49
7,tracy,70.01,136.46
8,john,67.90,112.37
9,simon,67.90,112.37


In [12]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,name,height,weight,BMI,weight_status
0,alex,1.67,51.25,18.38,underweight
1,ajay,1.82,61.91,18.69,normal
2,alice,1.76,69.41,22.41,normal
3,ravi,1.73,64.56,21.57,normal
4,joe,1.72,65.45,22.12,normal
5,jack,1.74,55.93,18.47,underweight
6,tom,1.77,64.18,20.49,normal
7,tracy,1.78,61.90,19.54,normal
8,john,1.72,50.97,17.23,underweight
9,simon,1.72,50.97,17.23,underweight


In [13]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [14]:
log("ETL Job Ended")